In [4]:
#do the essential imports
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
#utility functions
def init_weights(shape):
    return tf.Variable(tf.random_normal(shape, stddev=0.01))

In [6]:
#configuration variables
batch_size = 128
test_size  = 256

In [7]:
#read dataset
mnist              = input_data.read_data_sets("MNIST_data", one_hot=True)
trX, trY, teX, teY = mnist.train.images, mnist.train.labels, mnist.test.images, mnist.test.labels
#get batch of (28, 28, 1) for training images
trX = trX.reshape(-1, 28, 28, 1)
#get batch of (28, 28, 1) for testing images
teX = teX.reshape(-1, 28, 28, 1)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [20]:
#build the model
def model(X, w1, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden):
    #X->l1a->l1(maxpool)->dropout->l2a->l2(maxpool)->dropout->l3a->maxpool->
    
    
    #################################################################################################
    ######################LAYER-1 STARTS#############################################################
    #LAYER-1 INPUT: (?, 28, 28, 1) WEIGHTS: (3, 3, 1, 32) OUTPUT: (?, 14, 14, 32)
    #weights size:[3, 3, 1, 32])i.e. 32 outputs of convolutions of 3 X 3 X 1
    #OUTPUT:(?, 28, 28, 32); 
    l1a = tf.nn.relu(tf.nn.conv2d(X, w1, strides=[1, 1, 1, 1], padding='SAME'))
    #OUTPUT:(?, 14, 14, 32)
    l1  = tf.nn.max_pool(l1a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME') 
    l1  = tf.nn.dropout(l1, p_keep_conv)
    ####################LAYER 1 ENDS################################################################
    ################################################################################################
    
    
    ###############################################################################################
    #################LAYER 2 STARTS################################################################
    #LAYER-2 INPUT:(?, 14, 14, 32) WEIGHTS(3, 3, 32, 64) OUTPUT: (?, 7, 7, 64)
    #INPUT:(?, 14, 14, 32) OUTPUT:(?, 14, 14, 64)
    #weights size:[3, 3, 32, 64] i.e. 64 outputs of convolutions of 3 X 3 X 32
    l2a = tf.nn.relu(tf.nn.conv2d(l1, w2, strides=[1, 1, 1, 1], padding='SAME'))
    #INPUT:(?, 14, 14, 64) OUTPUT:(?, 7, 7, 64)
    l2  = tf.nn.max_pool(l2a, ksize=[1, 2, 2, 1], strides = [1, 2, 2, 1], padding='SAME')
    #do the dropout
    l2  = tf.nn.dropout(l2, p_keep_conv)
    #########################LAYER-2 ENDS HERE####################################################
    ##############################################################################################
    #LAYER-3 
    #INPUT:(?, 7, 7, 64) WEIGHTS(3, 3, 64, 128) OUTPUT:(?, 7, 7, 128)
    l3a = tf.nn.relu(tf.nn.conv2d(l2, w3, strides=[1, 1, 1, 1], padding='SAME'))
    #MAXPOOL of size (?, 4, 4, 128) 
    l3  = tf.nn.max_pool(l3a, ksize=[1, 2, 2, 1], strides=[1, 2, 2, 1], padding='SAME')
    #get the vector of size 4*4*128=2048
    l3  = tf.reshape(l3, [-1, w4.get_shape().as_list()[0]])
    l3  = tf.nn.dropout(l3, p_keep_conv)
    l4  =  tf.nn.relu(tf.matmul(l3, w4))
    l4  = tf.nn.dropout(l4, p_keep_hidden)
    pyx = tf.matmul(l4, w_o)
    return pyx

In [21]:
HIDDEN_FLATTENED_LAYER_SIZE = 625
OUTPUT_SIZE = 10
X = tf.placeholder("float", shape=[None, 28, 28, 1])
Y = tf.placeholder("float", shape=[None, OUTPUT_SIZE])
w1 = init_weights([3, 3, 1, 32])
w2 = init_weights([3, 3, 32, 64])
w3 = init_weights([3, 3, 64, 128])
w4 = init_weights([2048, HIDDEN_FLATTENED_LAYER_SIZE])
w_o = init_weights([HIDDEN_FLATTENED_LAYER_SIZE, OUTPUT_SIZE])
p_keep_conv   = tf.placeholder("float")
p_keep_hidden = tf.placeholder("float")
py_x          = model(X, w1, w2, w3, w4, w_o, p_keep_conv, p_keep_hidden)
cost          = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(labels=Y, logits=py_x))
train_op      = tf.train.RMSPropOptimizer(0.001, 0.9).minimize(cost)
predict_op    = tf.argmax(py_x, 1)

In [29]:
with tf.Session() as sess:
    tf.global_variables_initializer().run()
    for i in range(10):
        for start, end in zip(range(0, len(trX), batch_size), range(128, len(trX)+1, batch_size)):
#             print("Batch: " + str(start))
            sess.run(train_op, feed_dict={X:trX[start:end], Y:trY[start:end], p_keep_conv:0.8, p_keep_hidden:0.5})
        test_indices = np.arange(len(teX))
        np.random.shuffle(test_indices)
        test_indices = test_indices[:test_size]
        predictions  = sess.run(predict_op, feed_dict={X:teX[test_indices], Y:teY[test_indices], p_keep_conv:1.0, 
                                                      p_keep_hidden:1.0})
        actuals      = np.argmax(teY[test_indices], axis=1)
        accuracy     = np.mean(predictions==actuals)
        print("Epoch:- " + str(i) + " Accuracy:- " + str(accuracy))

Epoch:- 0 Accuracy:- 0.953125
Epoch:- 1 Accuracy:- 0.984375
Epoch:- 2 Accuracy:- 0.98046875
Epoch:- 3 Accuracy:- 0.984375
Epoch:- 4 Accuracy:- 0.984375
Epoch:- 5 Accuracy:- 0.984375
Epoch:- 6 Accuracy:- 0.99609375
Epoch:- 7 Accuracy:- 0.98046875
Epoch:- 8 Accuracy:- 0.99609375
Epoch:- 9 Accuracy:- 0.98828125
